In [ ]:
%load_ext autoreload
%autoreload 2

from querychains import Actor, OpenAiChatEngine, repeat_on_failure, ParsingFailure, Context, FileStorage, start_server
import json

In [ ]:

RPS = ["Rock", "Paper", "Scissors"]

def choose_winner(a1, a2):
    if a1 == a2:
        return None
    if a1 == "pass":
        return 1
    if a2 == "pass":
        return 0
    return int((RPS.index(a1) + 1) % 3 == RPS.index(a2)) 

assert choose_winner("Rock", "Paper") == 1
assert choose_winner("Rock", "Scissors") == 0
assert choose_winner("Paper", "Rock") == 0
assert choose_winner("Scissors", "Rock") == 1

In [ ]:
storage = FileStorage("data2")

In [ ]:

class RockLovingActor(Actor):

    def get_action(self, _prompt):
        return "Rock"


class LmRpsActor(Actor):

    def __init__(self, name, engine, ctx_meta=None,):
        super().__init__(name, ctx_meta)
        self.engine = engine

    def get_action(self, _prompt):
        history = "\n".join(f"Round {i + 1}: the opponent played {op}." for i, op in enumerate(self.observations()))
        parts = ["I'm playing game: Rock Paper Scissors."]
        if history:
            parts.append(f"The history of the game is the following: {history}\n\n")
        parts.append("What should I do in the next turn? Write only exactly one of following words: Rock, Paper, or Scissor as an answer")
        prompt = "\n".join(parts)

        def query():
            result = self.engine.query(prompt)
            result = result.strip()
            if result and result[-1] == ".":
                result = result[:-1]
            if result not in RPS:
                raise ParsingFailure("Invalid action")
            return result
            
        return repeat_on_failure(query, throw_if_fail=False, fail_value="pass")

engine = OpenAiChatEngine(temperature=1)
player1 = LmRpsActor("player1", engine, ctx_meta={"color": "#fcccbb"})
player2 = RockLovingActor("player2", ctx_meta={"color": "lightblue"})

with Context("game", storage=storage) as c:
    wins = [0, 0]
    for i in range(3):
        with Context(f"round {i}"):
            action1 = player1.act("play")
            action2 = player2.act("play")
            print(f"Round: {i} --> {action1} {action2}")
            winner = choose_winner(action1, action2)
            if winner is not None:
                wins[winner] += 1
            player1.observe(action2)
            player2.observe(action1)
c.uid            

In [ ]:
storage.start_server(port)

In [ ]:
with Context("time", storage=storage) as c:
    import time
    time.sleep(1)